<a href="https://colab.research.google.com/github/NandiniMehta0603/Parameter-Optimization-of-SVM/blob/main/Parameter_Optimization_of_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Machine Learning/Datasets/Skin_NonSkin.txt", names=['B', 'G', 'R', 'target'], sep="\t", index_col=False)
# df.head()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [19]:
df=pd.read_csv("/content/drive/MyDrive/ml/accelerometer df.csv",names=['target','speed percentage','x','y','z'])

In [20]:
df.head()

,target,speed percentage,x,y,z
0,1,20,1.004,0.090,-0.125
1,1,20,1.004,-0.043,-0.125
2,1,20,0.969,0.090,-0.121
3,1,20,0.973,-0.012,-0.137
4,1,20,1.000,-0.016,-0.121


In [22]:
# first_col = df.pop(df.columns[0])

# df[first_col.name] = first_col

# df.head()

In [14]:
# df = df.iloc[1:]

In [21]:
df.head()

,target,speed percentage,x,y,z
0,1,20,1.004,0.090,-0.125
1,1,20,1.004,-0.043,-0.125
2,1,20,0.969,0.090,-0.121
3,1,20,0.973,-0.012,-0.137
4,1,20,1.000,-0.016,-0.121


In [23]:
# train_accs = []
# test_accs = []
# svm_best_params = []

# for i in range(10):

#   print('-------- Sample {}-------\n'.format(i))
#   dfi = df.sample(frac=0.1)
#   X = dfi.iloc[:, :-1].values
#   y = dfi.iloc[:, -1].values
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)

#   sc = StandardScaler()
#   X_train = sc.fit_transform(X_train)
#   X_test = sc.transform(X_test)

#   # Optimize the SVM using grid search with cross-validation
#   param_grid = {
#       'C': [0.1, 1, 10, 100],
#       'gamma': [0.1, 0.01, 0.001, 0.0001],
#       'kernel': ['linear', 'rbf', 'sigmoid']
#   }
#   grid_search = GridSearchCV(SVC(), param_grid, cv=3, verbose=100, refit=True)
#   grid_search.fit(X_train, y_train)
#   best_params = grid_search.best_params_
#   svm_best_params.append(best_params)

#   print(best_params)
#   # Train and test the SVM with the best parameters
#   svm = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'], max_iter=1000)
#   svm.fit(X_train, y_train)
#   train_acc = svm.score(X_train, y_train)
#   test_acc = svm.score(X_test, y_test)
#   train_accs.append(train_acc)
#   test_accs.append(test_acc)

In [ ]:
# X = dfi.iloc[:, :-1].values
#   y = dfi.iloc[:, -1].values
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)

max_acc = 0
acc_list = []
param_list = []
for i in range(10):
    sample_df = df.sample(frac=0.40, replace=False)
    x_train, x_test, y_train, y_test = train_test_split(sample_df.drop('target', axis=1), sample_df.target, test_size=0.3, random_state=42, stratify=sample_df.target)

    x_train = StandardScaler().fit_transform(x_train)
    param_grid = { 
                'C':[0.1, 0.5, 1, 2, 3],
                'kernel':['rbf', 'poly', 'sigmoid', 'linear'],
                'degree':[1, 2, 3, 4],
                'gamma': [1, 0.1, 0.01, 0.001, 0.0001]
                }
    
    grid = GridSearchCV(SVC(max_iter=100), param_grid)
    grid.fit(x_train, y_train)  
    y_pred = grid.predict(x_test)
    print(grid.best_params_)
    print('Sample {}'.format(i+1))
    print('Accuracy: {}\n'.format(accuracy_score(y_test, y_pred))) 
    
    acc_list.append(accuracy_score(y_test, y_pred))
    param_list.append(grid.best_params_)
    
    if accuracy_score(y_test, y_pred) > max_acc:
        max_acc = accuracy_score(y_test, y_pred)
        best_params = grid.best_params_
        best_model = grid.best_estimator_
        best_sample_df = sample_df

In [ ]:
print(acc_list)
print(param_list)

In [ ]:
final_df = pd.DataFrame(data=[acc_list, param_list], index=['Accuracy', 'Parameters']).T
final_df

In [ ]:
print(max_acc)
print(best_params)
print(best_model)
best_sample_df.head()

In [ ]:
iterations = list()
acc_list = list()
x_train, x_test, y_train, y_test = train_test_split(best_sample_df.drop('salary', axis=1), best_sample_df.salary, test_size=0.3, random_state=42, stratify=best_sample_df.salary)
x_train = StandardScaler().fit_transform(x_train)

for i in range(100):
    x = i+1
    svm = SVC(C=2, degree=1, gamma=0.1, kernel='rbf', max_iter=x)
    svm.fit(x_train, y_train)
    y_pred = svm.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    acc_list.append(accuracy)
    iterations.append(x)

In [ ]:
iterations = np.array(iterations)
acc_list = np.array(acc_list)

In [ ]:
print(iterations.shape)
print(acc_list.shape)

In [ ]:
plt.plot(iterations, acc_list)
plt.xlabel('Number of iterations')
plt.ylabel('Accuracy')
plt.title('Convergence of SVC')
plt.show()